# Instroduction

This note will create dataset for calulation


In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_W")

dataset = "micost/user_crud_orpo"
login(token = hf_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Generate ramdon numbers for operations

In [3]:
import random

def calculate(op, A, B):
  if op == '+':
    return A + B
  elif op == '-':
    return A - B
  elif op == '*':
    return A * B
  elif op == '/':
    if B == 0:
      return "Division by zero"
    else:
      return A / B
  else:
    return "Invalid operator"

def generate_dataset(num_samples):
  dataset = []
  operators = ['+', '-', '*', '/']
  for _ in range(num_samples):
    op = random.choice(operators)
    A = random.randint(1, 100)
    B = random.randint(1, 100)
    C = random.randint(2, 10)
    result = calculate(op, A, B)
    wrong_result = calculate(op, result,C)
    prompt = f"{{'A':{A},'op':'{op}','B':{B}}}"
    correct_result = f"{{'result': '{result}'}}"
    wrong_result =  f"{{'result': '{wrong_result}'}}"
    chosen =[]
    rejected = []
    content = {"content": prompt, "role": "user"}
    answer = {"content": correct_result,"role": "assistant"}
    chosen.append(content)
    chosen.append(answer)
    answer = {"content": wrong_result,"role": "assistant"}
    rejected.append(content)
    rejected.append(answer)
    
    
    data = {
        "prompt": prompt,
        "chosen": chosen,
        "rejected":rejected
    }
    dataset.append(data)
  return dataset


data = generate_dataset(1000)


for item in data[:5]:
  print(item)

{'prompt': "{'A':44,'op':'-','B':1}", 'chosen': [{'content': "{'A':44,'op':'-','B':1}", 'role': 'user'}, {'content': "{'result': '43'}", 'role': 'assistant'}], 'rejected': [{'content': "{'A':44,'op':'-','B':1}", 'role': 'user'}, {'content': "{'result': '35'}", 'role': 'assistant'}]}
{'prompt': "{'A':96,'op':'*','B':66}", 'chosen': [{'content': "{'A':96,'op':'*','B':66}", 'role': 'user'}, {'content': "{'result': '6336'}", 'role': 'assistant'}], 'rejected': [{'content': "{'A':96,'op':'*','B':66}", 'role': 'user'}, {'content': "{'result': '44352'}", 'role': 'assistant'}]}
{'prompt': "{'A':68,'op':'-','B':17}", 'chosen': [{'content': "{'A':68,'op':'-','B':17}", 'role': 'user'}, {'content': "{'result': '51'}", 'role': 'assistant'}], 'rejected': [{'content': "{'A':68,'op':'-','B':17}", 'role': 'user'}, {'content': "{'result': '49'}", 'role': 'assistant'}]}
{'prompt': "{'A':33,'op':'/','B':14}", 'chosen': [{'content': "{'A':33,'op':'/','B':14}", 'role': 'user'}, {'content': "{'result': '2.357

In [4]:
from datasets import Dataset

dataset = Dataset.from_list(data)

# Push to huggingface

In [5]:
from datasets import load_dataset


dataset.push_to_hub(dataset)

HFValidationError: Repo id must be a string, not <class 'datasets.arrow_dataset.Dataset'>: 'Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})'.